# ByteNite Guide: Simple Demo
Launching a video encoding job via ByteNite API.

---
In this notebook, you will be launching a video encoding job by utilizing a sample video and a temporary storage bucket. A set of predefined encoding parameters will be used.

<h1 style="color:blue">Setup (edit with your info)</h1>

## Your ByteNite API Key ↓
Get it at https://app.bytenite.com/profile

In [1]:
# >> Paste here your API key (Go to app.bytenite.com/profile)
access_token = 'BYTENITE_ACCESS_TOKEN'

<h1 style="color:orange">Additional setup (edit if you need different encoding options)</h1>
The default encoding parameters for this notebook are:

- MP4 - H.264 slow - 480p - CRF 23

You can configure your parameters from the "New Job" wizard on app.bytenite.com, and then copy-paste the document by clicking on "Switch to JSON" on the top right.

_Note: "true" / "false" values must be changed to Python's "True" / "False" (capital)_

In [2]:
## Job parameters dictionary (JSON)
job_params_body={
  "data": {
    "output_code_template": "",
    "output_template": "{{.job_name}}_{{.output_params.Aspect.Resolution.Height}}",
    "outputs": [
      {
        "name": "",
        "output_params": {
          "aspect": {
            "cropping": {
              "bottom": 0,
              "left": 0,
              "right": 0,
              "top": 0
            },
            "orientation": {
              "flip": "",
              "rotation": ""
            },
            "padding": {
              "fill_color": "#000000",
              "final_aspect_ratio": ""
            },
            "resolution": {
              "aspect_ratio": "",
              "avoid_upscaling": False,
              "height": "480p"
            }
          },
          "audio": {
            "audio_bitrate": "192k",
            "audio_channel": "2.0",
            "audio_codec": "AAC",
            "audio_sample_rate": "48khz"
          },
          "video": {
            "bitrate": {
              "buffer_size": "1200k",
              "crf": 23,
              "rate_control_mode": "constant_quality"
            },
            "codec": "libx264",
            "codec_params": {
              "level": "",
              "preset": "slow",
              "profile": "",
              "tune": ""
            },
            "frame_rate": {
              "fps": "",
              "up_mode": "avoid"
            }
          }
        },
        "output_type": "mp4"
      }
    ]
  }
}



-------



<h1 style="color:red">Execution (do not alter)</h1>

In [3]:
# Imports
import string
import requests, os, time

<h2 style="color:red">Job creation and submission via ByteNite API</h2>

In [4]:
# JOBS CREATION

## Data source and data destination definition
input_video_url = "https://storage.googleapis.com/video-test-public/input/bbb.mp4"

data_source_body={
    "dataSource": {
    "dataSourceDescriptor": "url",
    "params": {
      "@type": "type.googleapis.com/bytenite.data_source.HttpDataSource",
      "name": "",
      "url": input_video_url,
      "username": "",
      "password": ""
    }
  },
    "dataDestination": {
        "dataSourceDescriptor": "bucket",
        "params": {}
  }
}

# Custom name
job_name = "Test_BBB_to_H264_SD"

# CREATE JOB
create_job_body = {"name": job_name, "templateId": "video-transcoding@v0.2"}
create_resp = requests.post('http://api.bytenite.com/v1/customer/jobs', json=create_job_body, headers={'Authorization': access_token}).json()
try:
    job_data = create_resp['job']
    job_id = job_data['id']
    print(f"Job \"{job_name}\" created - ID: {job_id}")
except:
    print(f"CREATE ERROR")
    print(create_resp)
               
# SET SOURCE AND DESTINATION
datasource_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/datasource/{job_id}', json=data_source_body, headers={'Authorization': access_token}).json()
try:
    job_data = datasource_resp['job']
except:
    print(f"DATASOURCE ERROR")
    print(datasource_resp)
    
# SET PARAMETERS
params_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/params/{job_id}', json=job_params_body, headers={'Authorization': access_token}).json()
try:
    job_data = params_resp['job']
except:
    print(f"PARAMETERS SETTING ERROR")
    print(params_resp)
        
time.sleep(0.5)
    
# RUN JOB
run_resp = requests.post(f'https://api.bytenite.com/v1/customer/jobs/run/{job_id}', json={}, headers={'Authorization': access_token}).json()
get_resp = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_id}', json={}, headers={'Authorization': access_token}).json()
try:
    job_started = run_resp['ok']
    job_data = get_resp['job']
    print(f"Job started!\n\n{job_data}")
except:
    print(f"RUN JOB ERROR")
    print(run_resp)


Job "Test_BBB_to_H264_SD" created - ID: f4ce2ca0-bf49-4fb7-a32f-da899167678e
Job started!

{'id': 'f4ce2ca0-bf49-4fb7-a32f-da899167678e', 'templateId': 'video-transcoding@v0.2', 'name': 'Test_BBB_to_H264_SD', 'config': {'minChunkSize': '0', 'maxChunkSize': '0', 'replicationFactor': 0, 'partitioningSettings': {}}, 'priority': 'EXECUTION_PRIORITY_UNDEFINED', 'startAt': None, 'deadline': None, 'submittedAt': None, 'startedAt': '2023-06-22T08:00:08.563491Z', 'completedAt': None, 'regions': None, 'state': 'JOB_STATE_PARTITIONING', 'dataSource': {'dataSourceDescriptor': 'url', 'params': {'@type': 'type.googleapis.com/bytenite.data_source.HttpDataSource', 'name': '', 'url': 'https://storage.googleapis.com/video-test-public/input/bbb.mp4', 'username': '', 'password': ''}}, 'params': {'schema': '', 'data': {'output_code_template': '', 'output_template': '{{.job_name}}_{{.output_params.Aspect.Resolution.Height}}', 'outputs': [{'name': '', 'output_params': {'aspect': {'cropping': {'bottom': 0, 'l

-------


<h1 style="color:green">Get the results (run after launching the jobs)</h1>

In [9]:
import dateutil.parser
from IPython.display import display, HTML

results_resp = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_id}/results', headers={'Authorization': access_token}).json()
print(f"{job_name}")
display(HTML(f"<a href='{input_video_url}' target='_blank'>Input: link</a>"))
try:
    results_data = results_resp['results']
    job_data = requests.get(f'https://api.bytenite.com/v1/customer/jobs/{job_id}', json=job_params_body, headers={'Authorization': access_token}).json()['job']
    job_time = (dateutil.parser.isoparse(job_data['completedAt']) - dateutil.parser.isoparse(job_data['startedAt'])).total_seconds()
    print(f"Running time: {int(job_time)} seconds")
    for j in range(max(len(results_data),1)):
        display(HTML(f"<a href='{results_data[j]['link']}' target='_blank'>Output {j+1}: link</a>"))
except KeyError:
    print(f"ERROR: {results_resp}")
except (IndexError, TypeError):
    display(HTML(f"<b>Not ready</b> (wait and rerun cell)..."))

Test_BBB_to_H264_SD


Running time: 67 seconds


## Troubleshooting

For more info, visit https://github.com/ByteNite2/bytenite-api-docs or contact us at support@bytenite.com